# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
from collections import Counter
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**
1. Все названия столбцов оформлены правильно
2. В столбцах days_employed и total_income есть пропуски. Пропускиесть одновременно в обоих столбцах, скорее всего, это техническая ошибка. Можно будет заполнить их средними по выделенным группам.
3. Тип данных в столбцах days_employed и total_income нужно заменить на int
4. В столбце days_employed даны как отрицательные, так и положительные значения, скорее всего отрицательные числа получились из-за метода подсчета, а вот положительные числа - очень большие, при их переводе из дней в года получается больше 1000 лет, эти данные некорректны. Необходимо привести числа к одному виду и избавиться от экстремальных значений.
6. В столбце dob_years есть возраст 0, такого быть не может, значит скорее всего это пропущенные значения. Необходимо заполнить их средними значениями для выделенных групп.
7. В столбце education значения не приведены к одному регистру.
8. Столбцы education и family_status можно вынести в отдельные словари, так как есть соответствующие им столбцы с числовым обозначением тех же категорий.
9. В столбце purpose цели взятия кредита указаны в свободной форме, а не разбиты по типам. С помощью лемматизации нужно выделить основные цели и проставить их в отдельном столбце.

## Шаг 2. Предобработка данных

### Обработка пропусков

Исследуем и посчитаем пропуски.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data[data['days_employed'].isna()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [6]:
data[data['dob_years'] == data['dob_years'].min()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [7]:
print('Нулевых значений в столбце dob_years',
      data[data['dob_years'] == data['dob_years'].min()]['dob_years'].count())

Нулевых значений в столбце dob_years 101


Пропуски встречаются в двух столбцах days_employed и total_income. В столбце dob_years встречаются нулевые значения, что также не может являться верным значением и скорее всего нулями обозначили пропуски.

#### Убираем нули в графе возраст

В датасете в столбце dob_years есть 101 нулевое значение.
Чтобы заполнить эти значения соответствующими числами, по возрасту можно разделить всех клиентов на три категории: пенсионеры, студенты и люди среднего возраста. Для каждой группы высчитать медианный возраст и присвоить его на место пропущенных значений.

Обрабатываем группу пенсионеры.

In [8]:
print('Пропущено',
      data[(data['income_type'] == 'пенсионер') & (
          data['dob_years'] == 0)]['dob_years'].count(),
      'значений')

Пропущено 20 значений


In [9]:
retiree_avg_age = int(data[(data['income_type'] == 'пенсионер') & (
    data['dob_years'] != 0)]['dob_years'].median())
print('Средний возраст пенсионеров -', retiree_avg_age, 'лет')

Средний возраст пенсионеров - 60 лет


In [10]:
data.loc[(data['income_type'] == 'пенсионер') & (
    data['dob_years'] == 0), 'dob_years'] = retiree_avg_age
print('Пропущено',
      data[(data['income_type'] == 'пенсионер') & (
          data['dob_years'] == 0)]['dob_years'].count(),
      'значений')

Пропущено 0 значений


В категории 'пенсионер' возраст проставлен.

Переходим к категории 'студент'.

In [11]:
print('Пропущено',
      data[(data['income_type'] == 'студент') & (
          data['dob_years'] == 0)]['dob_years'].count(),
      'значений')

Пропущено 0 значений


В категории 'студент' возраст проставлен.

Переходим к категории 'средний возраст'.

In [12]:
print('Пропущено',
      data[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'студент')
           & (data['dob_years'] == 0)]['dob_years'].count(),
      'значение')

Пропущено 81 значение


In [13]:
others_avg_age = int(data[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'студент')
                          & (data['dob_years'] != 0)]['dob_years'].median())
print('Средний возраст', others_avg_age, 'лет')

Средний возраст 39 лет


In [14]:
data.loc[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'студент')
         & (data['dob_years'] == 0), 'dob_years'] = others_avg_age

In [15]:
print('Пропущено',
      data[data['dob_years'] == 0]['dob_years'].count(),
      'значений')

Пропущено 0 значений


In [16]:
print('Минимальный возраст', data['dob_years'].min(), 'лет')

Минимальный возраст 19 лет


Новый минимальный возраст - 19 лет, все пропуски заполнены.

#### Убираем пропуски в столбце days_employed

Стаж как правило прямо пропорционален возрасту, поэтому для заполнения отстутствующих значений можно посчитать среднее соотношение возраста и стажа и высчитать стаж для каждого необходимого случая. Для этого сначала нужно привести все присутствующие значения к одному виду - положительному числу, выражающему стаж в днях. Отрицательные числа берем по модулю, а положительные делим на возраст из соответствующей строки. По видимости, вычисление стажа в таких случаях было сделано умножением среднего стажа на возраст, при этом в формулу попала ошибка и умножение на возраст было произведено два раза.

In [17]:
def correct_work_experience(row):
    """
    Функция приводит данные столбца days_employed к одному виду - 
    положительное число, выражающее стаж в днях.
    """
    days = row['days_employed']
    age = row['dob_years']
    if days == 'NaN':
        return days
    if days < 0:
        return abs(days)
    else:
        try:
            return days / age
        except:
            'Деление на ноль, в возрасте ошибка!'

In [18]:
data['days_employed'] = data.apply(correct_work_experience, axis=1)
max_work_exp = 100 * 365
print('В столбце days_employed',
      data[(data['days_employed'] > max_work_exp) | (
          data['days_employed'] < 0)]['days_employed'].count(),
      'некорректных значений')

В столбце days_employed 0 некорректных значений


Высчитываем средний стаж по возрастам и заполняем отсутствующие значения

In [19]:
avg_days_employed_by_age = data[data['days_employed']
                                > -1].groupby('dob_years')['days_employed'].mean()
avg_days_employed_by_age

dob_years
19     633.678086
20     684.944308
21     709.440930
22     868.335694
23     827.309437
24    1026.405485
25    1088.406453
26    1271.573924
27    1437.259400
28    1422.593250
29    1553.823200
30    1696.039355
31    1671.374840
32    1792.036251
33    1903.416062
34    2036.364071
35    2123.644038
36    2321.071632
37    2257.042401
38    2414.407934
39    2429.699261
40    2434.087248
41    2504.380391
42    2892.096726
43    2606.387229
44    2921.915095
45    2923.182144
46    3026.374616
47    3113.415705
48    3100.578211
49    3379.508378
50    3644.557522
51    3783.690103
52    4087.025165
53    4179.449681
54    4271.927614
55    4840.380138
56    4658.568610
57    5002.561283
58    4813.984153
59    5301.715962
60    5458.911446
61    5333.909322
62    5175.316770
63    5520.135585
64    5350.664472
65    5295.163648
66    5317.246312
67    5188.707321
68    5305.583720
69    5182.879012
70    5137.381396
71    4944.740105
72    5257.682559
73    4782.465685


In [20]:
days_employed_transformed = data.groupby(
    'dob_years')['days_employed'].transform('mean')
data['days_employed'] = data['days_employed'].fillna(days_employed_transformed)
print('В столбце days_employed',
      data[data['days_employed'].isna()]['days_employed'].count(),
      'пропусков')

В столбце days_employed 0 пропусков


#### Убираем пропуски в столбце total_income

Доход очень сложно высчитать исходя из имеющихся данных, поэтому для упрощения будем считать, что он зависит от типа занятости.

In [21]:
income_pivot = data.pivot_table(
    index='income_type', values='total_income', aggfunc='median')
income_pivot

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


In [22]:
income_transformed = data.groupby(
    'income_type')['total_income'].transform('mean')
data['total_income'] = data['total_income'].fillna(income_transformed)
print('В столбце total_income',
      data[data['total_income'].isna()]['total_income'].count(),
      'пропусков')

В столбце total_income 0 пропусков


Пропуски заполнены

**Вывод**

В исходной таблице пропуски были в трех столбцах dob_years, days_employed и total_income. С помощью разбиения на группы по возрасту или типу занятости и высчитывания соответствующей медианы мы заполнили все пропуски, данные готовы к дальнейшей обработке.

### Замена типа данных

В столбцах days_employed и total_income тип данных float с шестью знаками после запятой, что не соответствует формату данных, которые в них хранятся - дни и доходы в рублях, а также затрудняет восприятие данных. Необходимо заменить тип данных в этих столбцах на целочисленный.

In [23]:
data['days_employed'] = data['days_employed'].astype('int')

In [24]:
data['total_income'] = data['total_income'].astype('int')

In [25]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,6420,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

В столбцах days_employed и total_income заменили исходный тип данных float на целочисленный как более подходящий к формату хранимых данных.

### Исправление ошибок в данных

Проверим столбец children

In [26]:
data.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

В столбце встречаются некорректные значения -1 и 20. Скорее всего это ошибки, сделанные при введении информации человеком, и следует заменить их на 1 и 2 соответственно.

In [27]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
data.groupby('children')['children'].count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь данные корректны.

Посмотрим все уникальные значения в столбце education.

In [28]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Есть одни и те же слова написанные в разных регистрах, необходимо привести их к одному виду.

In [29]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверяем столбцы family_status и income_type.

In [30]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [31]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В столбцах family_status и income_type ошибок нет.

**Вывод**

В столбце children найдены и исправлены некорректные значения. Ошибочные написания были найдены только в столбце education и приведены к одному формату.

### Обработка дубликатов

Ищем дубликаты

In [32]:
print('В таблице', data.duplicated().sum(), 'полный дубликат')

В таблице 71 полный дубликат


Убираем дубликаты

In [33]:
data = data.drop_duplicates().reset_index(drop=True)
print('В таблице', data.duplicated().sum(), 'полных дубликатов')

В таблице 0 полных дубликатов


**Вывод**

В таблице был найден и удален 71 полный дубликат.

### Лемматизация

Для выделения основных целей кредита соберем все слова из столбца purpose и лемматизируем, а затем посчитаем самые популярные и на их основе произведем категоризацию.

In [34]:
text = ' '.join(data['purpose'])
lemmas = m.lemmatize(text)
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

Вручную выделяем цели взятия кредита и создаем словарь.
Слова 'покупка', 'приобретение', 'операция', 'сделка' для нас не значимы, так как могут относится и к недвижимости, и к автомобилю. Прилагательные (кроме 'коммерческий' и 'жилой'), глаголы и предлоги также могут относиться к разным целям. Из оставшихся существительных выделяются слова 'свадьба', 'образование', 'автомобиль', 'ремонт', а также несколько слов относящихся к недвижимости - 'жилье', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'сдача'. Я разбила недвижимость на жилую и коммерческую, так как возврат кредита на коммерческую недвижимость будет зависеть от прибыли, которую получит собственник с этой недвижимости, а кредит на жилую недвижимость будет выплачиваться только из собственных средств. Возможно было также выделить цели 'свадьба' и 'ремонт' в категорию 'потребительский кредит', но в связи с характером трат (свадебный кредит частично возмещается подарками от гостей) и возврат по кредитам может быть разный.

In [35]:
purpose_types = [[0, 'образование'],
                 [1, 'свадьба'],
                 [2, 'автомобиль'],
                 [3, 'ремонт'],
                 [4, 'коммерческая недвижимость'],
                 [5, 'жилая недвижимость'],
                 [6, 'другое']]
purpose_columns = ['purpose_type_id', 'purpose_type']
purpose_type_dict = pd.DataFrame(data=purpose_types, columns=purpose_columns)
purpose_type_dict

,purpose_type_id,purpose_type
0,0,образование
1,1,свадьба
2,2,автомобиль
3,3,ремонт
4,4,коммерческая недвижимость
5,5,жилая недвижимость
6,6,другое


In [36]:
def purpose_classification(words):
    if 'образование' in m.lemmatize(words):
        return 0
    if 'свадьба' in m.lemmatize(words):
        return 1
    if 'автомобиль' in m.lemmatize(words):
        return 2
    if 'ремонт' in m.lemmatize(words):
        return 3
    if 'жилье' in m.lemmatize(words) or 'недвижимость' in m.lemmatize(words) or 'строительство' in m.lemmatize(words):
        if 'коммерческий' in m.lemmatize(words) or 'сдача' in m.lemmatize(words):
            return 4
        else:
            return 5
    return 6

In [37]:
data['purpose_type_id'] = data['purpose'].apply(purpose_classification)

In [38]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0
4,0,6420,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,5
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,5
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,1
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5


**Вывод**

С помощью лемматизации мы выделили основные цели взятия кредита, составили с ними словарь и проставили в новый столбец соответствующее числовое обозначение.

### Категоризация данных

Первый словарь мы создали при присваивании типов кредитов.
Также в таблице можно выделить следующие словари:
1. Семейное положение
2. Образование

In [39]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.sort_values(by='family_status_id')

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [40]:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.sort_values(by='education_id')

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [41]:
data = data.drop('family_status', axis=1)
data = data.drop('education', axis=1)
data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type_id
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,5
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,5
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,0
4,0,6420,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,1
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,5
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,5
7,0,152,50,1,0,M,сотрудник,0,135823,образование,0
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,1
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,5


**Вывод**

В таблице выделены три словаря:
1. Цель взятия кредита
2. Семейное положение
3. Образование

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [42]:
debt_childless = data[data['children'] == 0]['debt'].sum(
) / data[data['children'] == 0]['debt'].count()
print(f'Клиенты без детей имели задолженность в {debt_childless:.2%} случаев')

Клиенты без детей имели задолженность в 7.54% случаев


In [43]:
debt_children = data[data['children'] != 0]['debt'].sum(
) / data[data['children'] != 0]['debt'].count()
print(f'Клиенты с детьми имели задолженность в {debt_children:.2%} случаев')

Клиенты с детьми имели задолженность в 9.21% случаев


In [44]:
debt_child = data.pivot_table(
    index='children', columns='debt', values='gender', aggfunc='count')
debt_child['debtor_per_person'] = debt_child[1] / \
    (debt_child[1] + debt_child[0])
debt_child

debt,0,1,debtor_per_person
children,,,
0,13028.0,1063.0,0.075438
1,4410.0,445.0,0.091658
2,1926.0,202.0,0.094925
3,303.0,27.0,0.081818
4,37.0,4.0,0.097561
5,9.0,NaN,NaN


**Вывод**

Вероятно клиенты с детьми более склоны задерживаться с выплатой кредита, однако это может быть статистическая погрешность.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
debt_family_status = data.pivot_table(
    index='family_status_id', columns='debt', values='gender', aggfunc='count')
debt_family_status['debtor_per_person'] = debt_family_status[1] / \
    (debt_family_status[1] + debt_family_status[0])
debt_family_status = debt_family_status.sort_values(by='debtor_per_person')
debt_family_status = debt_family_status.merge(
    family_status_dict, on='family_status_id', how='left')
debt_family_status

,family_status_id,0,1,debtor_per_person,family_status
0,2,896,63,0.065693,вдовец / вдова
1,3,1110,85,0.071130,в разводе
2,0,11408,931,0.075452,женат / замужем
3,1,3763,388,0.093471,гражданский брак
4,4,2536,274,0.097509,Не женат / не замужем


**Вывод**

Люди, не желающие брать на себя брачные обязательства, и к обязательствам по кредиту относятся так себе.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
median_income = data['total_income'].median()
low_income = data[data['total_income'] <
                  median_income]['total_income'].median()
high_income = data[data['total_income'] >=
                   median_income]['total_income'].median()

In [47]:
def income_type(income):
    if income < low_income:
        return 'низкий доход'
    if income < high_income:
        return 'средний доход'
    return 'высокий доход'


data['income'] = data['total_income'].apply(income_type)

In [48]:
debt_income = data.pivot_table(
    index='income', columns='debt', values='gender', aggfunc='count')
debt_income['debtor_per_person'] = debt_income[1] / \
    (debt_income[1] + debt_income[0])
debt_income

debt,0,1,debtor_per_person
income,,,
высокий доход,5035,375,0.069316
низкий доход,4936,427,0.079620
средний доход,9742,939,0.087913


**Вывод**

Люди с высоким доходом лучше всего возвращают кредиты.

- Как разные цели кредита влияют на его возврат в срок?

In [49]:
debt_purpose = data.pivot_table(
    index='purpose_type_id', columns='debt', values='gender', aggfunc='count')
debt_purpose['debtor_per_person'] = debt_purpose[1] / \
    (debt_purpose[1] + debt_purpose[0])
debt_purpose = debt_purpose.sort_values(by='debtor_per_person')
debt_purpose = debt_purpose.merge(
    purpose_type_dict, on='purpose_type_id', how='left')
debt_purpose

,purpose_type_id,0,1,debtor_per_person,purpose_type
0,3,572,35,0.057661,ремонт
1,5,7646,596,0.072313,жилая недвижимость
2,4,1811,151,0.076962,коммерческая недвижимость
3,1,2138,186,0.080034,свадьба
4,0,3643,370,0.092200,образование
5,2,3903,403,0.093590,автомобиль


**Вывод**

Лучше всего возвращают кредиты на ремонт, а вот образовательные и автокредиты возвращать не спешат.

## Шаг 4. Общий вывод

По результатам обработки данных, семейное положение и наличие детей действительно влияют на возврат кредита в срок. При этом семейные, но бездетные люди с высоким доходом, берущие кредит на ремонт, с наибольшей вероятностью вернут его вовремя, тогда как холостяки с детьми и средним доходом, берущие автокредит, с большей вероятностью выплаты задержат.